# 역전파 학습 알고리즘의 구현과 실험

## 설치

In [ ]:
!pip install pillow # png 파일을 불러오기 위해 사용

## import

In [154]:
import random
import math
from PIL import Image

## 파라미터

In [191]:
input_size=16*16 # 16 x 16 픽셀
output_size = 7 # t, u, v, w, x, y, z 7개 
hidden_size = 1000
learning_cycle = 100 # 학습 사이클 횟수 
data_set = 10 # 데이터 셋의 개수
test_set = 20 # 추론시 사용하는 데이터 셋의 개수
learning_rate = 0.05 # 학습률

## 신경망 구현  
input_layer: 16 x 16 = 256개  
hidden_layer1: 1000개  
hidden_layer2: 1000개  
output_layer: 7개  

In [192]:
input_layer = [0]*input_size # input_layer를 만들어준다 
output_layer = [0]*output_size # output_layer를 만들어준다 
hidden_layer1 = [0]*hidden_size #hidden_layer1을 만들어준다
hidden_layer2 = [0]*hidden_size #hidden_layer2를 만들어준다


w1 = [[0 for col in range(input_size)]for row in range(hidden_size)] # 첫번째 가중치 (input_layer에서 hidden_layer로 가는 가중치)
w2 = [[0 for col in range(hidden_size)]for row in range(hidden_size)] # 두번째 가중치 (hidden_layer1에서 hidden_layer2로 가는 가중치)
w3 = [[0 for col in range(hidden_size)]for row in range(output_size)] # 세번째 가중치 (hidden_layer2에서 output_layer로 가는 가중치)

## 가중치 초기화 

In [193]:
# w1 초기화
for i in range(hidden_size): # 행의 개수만큼 for문을 돌려준다 
    for j in range(input_size): # 열의 개수만큼 for문을 돌려준다 
        w1[i][j]=float(round(random.random(),4)) # i행 j열의 값을 0이상 1미만의 랜덤값으로 만들어준다 소수점 4자리까지 표시(반올림해준다)

# w2 초기화
for i in range(hidden_size): # 행의 개수만큼 for문을 돌려준다 
    for j in range(hidden_size): # 열의 개수만큼 for문을 돌려준다 
        w2[i][j]=float(round(random.random(),4)) # i행 j열의 값을 0이상 1미만의 랜덤값으로 만들어준다 소수점 4자리까지 표시(반올림해준다)

# w3 초기화
for i in range(output_size): # 행의 개수만큼 for문을 돌려준다 
    for j in range(hidden_size): # 열의 개수만큼 for문을 돌려준다 
        w3[i][j]=float(round(random.random(),4)) # i행 j열의 값을 0이상 1미만의 랜덤값으로 만들어준다 소수점 4자리까지 표시(반올림해준다)


## 행렬곱 함수

In [194]:
def matrixmult(A, B): 
    row_A = len(A)
    col_A = len(A[0])
    row_B = len(B) # B는 input 값이기 떄문에 항상 열 개수가 1이다
    
    if col_A != row_B:
        raise ValueError("행렬 곱셈이 불가능합니다")
    
    C=[0]*row_A # C의 크기는 A의 행 수 x B의 열 수 인데 B의 열 수는 1이라 제외 
    for i in range(row_A):
            for k in range(col_A):
                C[i] += A[i][k]*B[k] # 행렬곱 c[i][j] += A[i][k]*B[k][j](j는 B의 열 번호) 이지만 여기서는 B의 열 개수가 항상 1이므로 제외
    return C

## 전치 함수

In [195]:
def transeposed(a):
    t = list(zip(*a))
    return t

## 이미지 함수

In [196]:
def img_processing(alphabet,num,check): # 마지막 입력이 0이면 학습 1이면 추론으로 본다
    if(check==0):
        img_path = str('./image/learning/'+(chr(alphabet+ord('t')))+str(num+1)+'.png') 
    else:
        img_path = str('./image/input/'+(chr(alphabet+ord('t')))+str(num+1)+'.png') 
    img = Image.open(img_path) # 이미지를 불러온다
    img = img.convert("L") # 이미지를 흑백으로 바꾼다 (0~255의 값을 가진다)
    img = list(img.getdata()) # 이미지 데이터를 list 형태로 바꾸어준다 
    
    for i in range(len(img)): # img가 0~1사이의 값을 가질 수 있도록 정규화 해준다 
        img[i] = img[i]/255 
    
    return img

## 순전파 함수

In [197]:
def FP(img): 
    #### input_layer ####
    for i in range(input_size):
        input_layer[i] = float(img[i]) # input_layer에 이미지를 넣어준다
    
    #### hidden_layer1 ####
    hidden_input1 = matrixmult(w1, input_layer) # 행렬곱을 해준다 (가중치 계산 input_layer -> hidden_layer1)
            
    for i in range(hidden_size):
        hidden_layer1[i] = hidden_input1[i] # hidden_layer1에 행렬곱한 결과를 넣어준다 
            
    hidden_output1 = [0]*hidden_size # hidden_layer1의 output을 초기화 해준다 
    for i in range(hidden_size): 
        hidden_output1[i] = max(0,hidden_layer1[i]) # ReLU연산을 해준다음 hidden_output1에 저장해준다 
    
    #### hidden_layer2 ####
    hidden_input2 = matrixmult(w2, hidden_output1) # 행렬곱을 해준다 (가중치 계산 hidden_layer1 -> hidden_layer2)
    
    for i in range(hidden_size):
        hidden_layer2[i]=hidden_input2[i] # hidden_layer2에 행렬곱한 결과를 넣어준다 
        
    hidden_output2 = [0]*hidden_size # hidden_layer2의 output을 초기화 해준다 
    for i in range(hidden_size):
        hidden_output2[i]=max(0,hidden_layer2[i]) # ReLU연산을 해준다음 hidden_output2에 저장해준다 
    
    #### output_layer ####
    output_input = matrixmult(w3, hidden_output2) # 행렬곱을 해준다 (가중치 계산 hidden_layer2 -> output_layer)
    
    for i in range(output_size):
        output_layer[i]=output_input[i] # output_layer에 행렬곱한 결과를 넣어준다 
        
    output_output = [0]*output_size # output_layer의 output을 초기화 해준다
    for i in range(output_size):
        output_output[i]=max(0,output_layer[i])
        
    return hidden_output1, hidden_output2, output_output
    
    
    

## 오차 역전파 함수

In [198]:
def BP(output, target):
    
    output_error=[0]*output_size
    for i in range(output_size):
        if i == target: # 만약 x가 target과 같다면 목표값을 1로 설정해준다
            output_error[i] = float(1 - output[i]) # 오차 = 목표값 - 결과값 
        else:    # target과 다르다면 목표값을 0으로 설정해준다 
            output_error[i] = float(0 - output[i]) # 오차 = 목표값 - 결과값
            
    hidden_error2 = matrixmult(transeposed(w3), output_error) # 가중치 행렬을 전치하게되면 역전파 행렬이 된다 
    hidden_error1 = matrixmult(transeposed(w2), hidden_error2)
    input_error = matrixmult(transeposed(w1), hidden_error1)
    
    return input_error, hidden_error1, hidden_error2, output_error
    

## 가중치 업데이트

In [199]:
def update(hidden_output1, hidden_output2, hidden_error1, hidden_error2, output_error):
    # 가중치 업데이트값 초기화
    dw1 = [[0 for col in range(input_size)]for row in range(hidden_size)] # 첫번째 가중치 (input_layer에서 hidden_layer로 가는 가중치)
    dw2 = [[0 for col in range(hidden_size)]for row in range(hidden_size)] # 두번째 가중치 (hidden_layer1에서 hidden_layer2로 가는 가중치)
    dw3 = [[0 for col in range(hidden_size)]for row in range(output_size)] # 세번째 가중치 (hidden_layer2에서 output_layer로 가는 가중치)

    # 가중치 행번호: 순전파 일때 가중치가 도착하는 레이어 번호
    # 가중치 열번호: 순전파 일때 가중치가 출발하는 레이어 번호
    # w3 가중치 업데이트
    for i in range(output_size):
        for j in range(hidden_size):
            de_relu = (output_layer[i]>0)*output_error[i]  # 행 번호가 가중치가 도착하는 레이어의 번호이기 때문에 행 번호를 사용한다
            dw3[i][j] = learning_rate * de_relu * hidden_output2[j] # 열 번호
            w3[i][j] = w3[i][j] + dw3[i][j]
   
    # w2 가중치 업데이트
    for i in range(hidden_size):
        for j in range(hidden_size):
            de_relu = (hidden_layer2[i]>0)*hidden_error2[i] # 행 번호가 가중치가 도착하는 레이어의 번호이기 때문에 행 번호를 사용한다
            dw2[i][j] = learning_rate * de_relu * hidden_output1[j]
            w2[i][j] = w2[i][j] + dw2[i][j]
            
    # w1 가중치 업데이트
    for i in range(hidden_size):
        for j in range(input_size):
            de_relu = (hidden_layer1[i]>0)*hidden_error1[i] # 행 번호가 가중치가 도착하는 레이어의 번호이기 때문에 행 번호를 사용한다
            dw1[i][j] = learning_rate * de_relu * input_layer[j]
            w1[i][j] = w1[i][j] + dw1[i][j]
            
    return 

## 학습 함수

In [200]:
def practice():
    for i in range(learning_cycle): # 학습 사이클 
        print(str(i) + 'cycle') # cycle 수를 출력한다 
        for j in range(output_size): # t~z 까지 
            for k in range(data_set): # 데이터셋의 개수만큼 반복
                # 이미지 가공 
                img = img_processing(j, k, 0)
                hidden_output1, hidden_output2, output_output = FP(img)
                input_error, hidden_error1, hidden_error2, output_error = BP(output_output, j)
                update(hidden_output1, hidden_output2, hidden_error1, hidden_error2, output_error)


## 추론 함수

In [201]:
def inference():
    for i in range(output_size): # t~z 까지
        for j in range(test_set): # 데이터셋의 개수만큼 반복
            img = img_processing(i, j, 1)
            output_output = FP(img)
            output = output_output.index(max(output_output)) # output_output에서 가장 큰값의 인덱스 번호 
            print('파일명: '+chr(i+ord('t'))+str(j)+ ' 추론값: ' + chr(output+ord('t')))

In [202]:
def main():
    practice()
    #inference()

In [203]:
if __name__ == "__main__":
    main()

0cycle


In [204]:
print(j)

299


In [206]:
hidden_layer1

[-1759853935.228185,
 -1832887071.493226,
 -1796285406.3186483,
 -1838926412.671419,
 -1834831111.874755,
 -1861099707.021084,
 -1886831507.47401,
 -1901144333.6675966,
 -1808806652.8529325,
 -1797927849.4121346,
 -1903170578.783208,
 -1883479700.07763,
 -1884195603.0886314,
 -1914393921.251168,
 -1886237221.143505,
 -1861089294.4907317,
 -1882094995.2642765,
 -1876667832.659534,
 -1839421716.1834552,
 -1891316877.3754191,
 -1718078268.3705895,
 -1938439332.306066,
 -1796998203.1668844,
 -1824727179.4589884,
 -1721839642.020689,
 -1884109041.162937,
 -1861646993.459056,
 -1845663156.9652894,
 -1799763890.2130594,
 -1879361262.1440792,
 -1919285986.1765437,
 -1916571639.5417907,
 -1838417149.0406559,
 -1878638009.2991962,
 -1889934256.1552103,
 -1859751982.0845335,
 -1809555382.0441494,
 -1910694634.322243,
 -1826192125.2854111,
 -1865254875.7332916,
 -1850807985.5796652,
 -1846006567.334351,
 -2008090336.5429456,
 -1963300739.4776077,
 -1934953241.6852446,
 -1698814171.026201,
 -175074